## Convert filter_blast function to use pandas

In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys
sys.path.append('/Users/hcross/analysis/git_projects/genbanking/gb_parsers/')
from blast_parsers import blast_parser

In [4]:
# now try to re-parse the small file
blast_tab = blast_parser('example_blast_output_default_format.txt')

In [5]:
blast_tab.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,MA_19953_MA_19953.g2.t1,gi|71153422|sp|Q9LLL8.1|SBT4E_ARATH,35.537,121,65,5,280,394,348,461,8.070000e-07,54.7
1,MA_19953_MA_19953.g3.t1,gi|60392921|sp|P11369.2|LORF2_MOUSE,37.681,69,42,1,209,276,510,578,3.920000e-05,50.1
2,MA_19953_MA_19953.g4.t1,gi|141475|sp|P14381.1|YTX2_XENLA,22.966,762,480,22,328,1012,128,859,2.240000e-35,150.0
3,MA_19953_MA_19953.g5.t1,gi|81615471|sp|Q6LU24.1|PUR4_PHOPR,31.868,91,57,2,13,98,971,1061,2.400000e-02,40.0
4,MA_19953_MA_19953.g6.t1,gi|81960279|sp|Q914H8.1|Y054_SIFVH,19.880,166,122,4,17,175,176,337,9.800000e-02,37.7


### Build a function that includes the variables from original, but adding pandas function

#### Use commands like these:

```python
blast2 = blast_table.loc[blast_table['pident'] > 35]

blast3 = blast_table[(blast_table['pident'] > 35) & (blast_table['length'] > 100)]
```


In [10]:
great = '>'
blast2 = blast_tab.loc[blast_tab['pident'] %s 35] % (great)

SyntaxError: invalid syntax (<ipython-input-10-c8530eaa4353>, line 2)

In [11]:
string = "blast2 = blast_tab.loc[blast_tab['pident'] > 35]"
exec(string)

In [12]:
blast2.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,MA_19953_MA_19953.g2.t1,gi|71153422|sp|Q9LLL8.1|SBT4E_ARATH,35.537,121,65,5,280,394,348,461,8.070000e-07,54.7
1,MA_19953_MA_19953.g3.t1,gi|60392921|sp|P11369.2|LORF2_MOUSE,37.681,69,42,1,209,276,510,578,3.920000e-05,50.1
5,MA_19953_MA_19953.g7.t1,gi|130582|sp|P10978.1|POLX_TOBAC,64.103,39,14,0,114,152,530,568,2.210000e-07,55.1
6,MA_19953_MA_19953.g7.t1,gi|130582|sp|P10978.1|POLX_TOBAC,50.000,40,20,0,52,91,295,334,2.000000e-03,43.5
9,MA_19953_MA_19953.g9.t1,gi|135915|sp|P28493.1|PR5_ARATH,65.948,232,75,4,1,232,12,239,5.910000e-99,294.0


**In order to be flexible with testing fields, have to create strings, and then convert it to command using exec(command_string)** 
found here:


http://stackoverflow.com/questions/701802/how-do-i-execute-a-string-containing-python-code-in-python

In [9]:
def filter_blast(blastDF, blast_parameters):
    """Filter a blast result dictionary based on variable parameters using pandas"""
    ## >> have to add more to following
    field_match = {'SUB_LENGTH':'slen','MIN_QSTART':'qstart','MAX_QSTART':'qstart','MIN_QEND':'qend','MAX_QEND':'qend','MIN_SEND':'send','MAX_SEND':'send','MIN_BITSCORE':'bitscore','MIN_POSITIVE':'positive','MAX_GAPOPEN':'gapopen','MAX_GAPS':'gaps','IN_TITLE':'stitle','IN_SSEQ':'sseq','MIN_SSTART':'sstart','MAX_SSTART':'sstart','MIN_QCOV':'qcovs','MIN_LENGTH':'length','MIN_PCT_ID':'pident','MAX_MISMATCH':'mismatch','MAX_EVALUE':'evalue','IN_SUBJECT_ID':'sseqid'}
    mins = ['MIN_LENGTH','MIN_PCT_ID','MIN_SSTART','MIN_QCOV','SUB_LENGTH','MIN_QSTART','MIN_QEND','MIN_SEND','MIN_BITSCORE','MIN_POSITIVE'] 
    maxes = ['MAX_MISMATCH','MAX_EVALUE','MAX_SSTART','MAX_QSTART','MAX_QEND','MAX_SEND','MAX_GAPOPEN','MAX_GAPS'] 
    containers = ['IN_SUBJECT_ID','IN_TITLE','IN_SSEQ']
    filtered_blast = {}
    blast_parameters.pop('FORMAT')
    num_conditions = len(blast_parameters)
    # for now have to build it as a string and then run 
    string_list = []
    for k,v in blast_parameters.iteritems():
        # blast2 = blast_table.loc[blast_table['pident'] > 35]
        blast_field = field_match[k]
        string = "blastDF[blast_field]"
        if k in mins:
            #string = string + "> "+v+"]"
            string = string + "> %s" %(v)
            #if hit_dict[blast_field] > v:   
        elif k in maxes:
            string = string + "< %s" %(v)
            #if hit_dict[blast_field] < v:
                
        elif k in containers:
            # bb5 = bb[bb['stitle'].str.contains('Maireana')]
            #string = string+".str.contains("+v+")]"
            string = string+".str.contains('%s')" % (v)
            #if v in hit_dict[blast_field]: # need to fix this, was in v
        string_list.append(string)
    if num_conditions == 1:
        cmd_string = string_list[0]
        final_string = "filt_blastDF = blastDF["+cmd_string+"]"
        exec(final_string)
        
    # for multiple conditions:
    #blast3 = blast_table[(blast_table['pident'] > 35) & (blast_table['length'] > 100)]
    else:
        initial = "filt_blastDF = blastDF["
        for cmd in string_list:
            new_string = "("+cmd+")"
            initial = initial+new_string+"&"
        semifinal = initial[:-1]
        final = semifinal+"]"
        exec(final)
    return filt_blastDF
    

In [85]:
# try a simple one 
para_dict = {'MIN_LENGTH':100, 'FORMAT':'default'}

In [86]:
filt1 = filter_blast(blast_tab, para_dict)

In [87]:
filt1.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,MA_19953_MA_19953.g2.t1,gi|71153422|sp|Q9LLL8.1|SBT4E_ARATH,35.537,121,65,5,280,394,348,461,8.070000e-07,54.7
2,MA_19953_MA_19953.g4.t1,gi|141475|sp|P14381.1|YTX2_XENLA,22.966,762,480,22,328,1012,128,859,2.240000e-35,150.0
4,MA_19953_MA_19953.g6.t1,gi|81960279|sp|Q914H8.1|Y054_SIFVH,19.880,166,122,4,17,175,176,337,9.800000e-02,37.7
9,MA_19953_MA_19953.g9.t1,gi|135915|sp|P28493.1|PR5_ARATH,65.948,232,75,4,1,232,12,239,5.910000e-99,294.0
13,MA_10432594_MA_10432594.g13.t1,gi|134034966|sp|Q99315.3|YG31B_YEAST,28.252,1207,672,27,418,1453,267,1450,8.210000e-132,455.0


In [64]:
para_dict2 = {'MAX_MISMATCH':30, 'FORMAT':'default'}

In [65]:
filt2 = filter_blast(blast_tab, para_dict2)

filt_blastDF = blastDF[blastDF[blast_field]< 30]


In [66]:
filt2.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
5,MA_19953_MA_19953.g7.t1,gi|130582|sp|P10978.1|POLX_TOBAC,64.103,39,14,0,114,152,530,568,2.210000e-07,55.1
6,MA_19953_MA_19953.g7.t1,gi|130582|sp|P10978.1|POLX_TOBAC,50.000,40,20,0,52,91,295,334,2.000000e-03,43.5
11,MA_10434223_MA_10434223.g11.t1,gi|296439825|sp|A7MC48.2|SHSA9_XENTR,42.857,28,16,0,116,143,1,28,5.000000e+00,30.4
12,MA_10434223_MA_10434223.g12.t1,gi|11135132|sp|O65049.1|TRXH_PICMA,98.980,98,1,0,18,115,28,125,4.660000e-63,194.0
19,MA_10432594_MA_10432594.g18.t1,gi|75215641|sp|Q9XI47.1|SHH1_ARATH,38.462,26,15,1,3,27,65,90,5.200000e+00,28.5


In [88]:
para_dict3 = {'IN_SUBJECT_ID': 'ARATH', 'FORMAT':'default'}

In [89]:
filt3 = filter_blast(blast_tab, para_dict3)

In [90]:
filt3.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,MA_19953_MA_19953.g2.t1,gi|71153422|sp|Q9LLL8.1|SBT4E_ARATH,35.537,121,65,5,280,394,348,461,8.070000e-07,54.7
9,MA_19953_MA_19953.g9.t1,gi|135915|sp|P28493.1|PR5_ARATH,65.948,232,75,4,1,232,12,239,5.910000e-99,294.0
17,MA_10432594_MA_10432594.g17.t1,gi|75154272|sp|Q8L817.1|ATCA7_ARATH,51.685,178,85,1,44,220,36,213,2.070000e-62,206.0
18,MA_10432594_MA_10432594.g17.t1,gi|75154272|sp|Q8L817.1|ATCA7_ARATH,45.038,131,54,1,219,331,143,273,2.070000e-30,121.0
19,MA_10432594_MA_10432594.g18.t1,gi|75215641|sp|Q9XI47.1|SHH1_ARATH,38.462,26,15,1,3,27,65,90,5.200000e+00,28.5


In [35]:
string2 = "blast3 = blast_tab[blast_tab['sseqid'].str.contains('ARATH')]"
exec(string2)

In [36]:
blast3.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,MA_19953_MA_19953.g2.t1,gi|71153422|sp|Q9LLL8.1|SBT4E_ARATH,35.537,121,65,5,280,394,348,461,8.070000e-07,54.7
9,MA_19953_MA_19953.g9.t1,gi|135915|sp|P28493.1|PR5_ARATH,65.948,232,75,4,1,232,12,239,5.910000e-99,294.0
17,MA_10432594_MA_10432594.g17.t1,gi|75154272|sp|Q8L817.1|ATCA7_ARATH,51.685,178,85,1,44,220,36,213,2.070000e-62,206.0
18,MA_10432594_MA_10432594.g17.t1,gi|75154272|sp|Q8L817.1|ATCA7_ARATH,45.038,131,54,1,219,331,143,273,2.070000e-30,121.0
19,MA_10432594_MA_10432594.g18.t1,gi|75215641|sp|Q9XI47.1|SHH1_ARATH,38.462,26,15,1,3,27,65,90,5.200000e+00,28.5


In [91]:
# now try with multis
para_dict4 = {'MIN_LENGTH':10, 'MAX_MISMATCH':50, 'FORMAT':'default'}

In [92]:
filt4 = filter_blast(blast_tab, para_dict4)

In [93]:
filt4.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
1,MA_19953_MA_19953.g3.t1,gi|60392921|sp|P11369.2|LORF2_MOUSE,37.681,69,42,1,209,276,510,578,3.920000e-05,50.1
5,MA_19953_MA_19953.g7.t1,gi|130582|sp|P10978.1|POLX_TOBAC,64.103,39,14,0,114,152,530,568,2.210000e-07,55.1
6,MA_19953_MA_19953.g7.t1,gi|130582|sp|P10978.1|POLX_TOBAC,50.000,40,20,0,52,91,295,334,2.000000e-03,43.5
7,MA_19953_MA_19953.g7.t1,gi|130582|sp|P10978.1|POLX_TOBAC,30.189,53,37,0,1,53,112,164,4.400000e+00,32.3
10,MA_10434223_MA_10434223.g10.t1,gi|146291076|sp|Q7LHG5.2|YI31B_YEAST,31.667,60,39,1,231,290,663,720,4.830000e-04,45.1


In [80]:
blast_tab.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,MA_19953_MA_19953.g2.t1,gi|71153422|sp|Q9LLL8.1|SBT4E_ARATH,35.537,121,65,5,280,394,348,461,8.070000e-07,54.7
1,MA_19953_MA_19953.g3.t1,gi|60392921|sp|P11369.2|LORF2_MOUSE,37.681,69,42,1,209,276,510,578,3.920000e-05,50.1
2,MA_19953_MA_19953.g4.t1,gi|141475|sp|P14381.1|YTX2_XENLA,22.966,762,480,22,328,1012,128,859,2.240000e-35,150.0
3,MA_19953_MA_19953.g5.t1,gi|81615471|sp|Q6LU24.1|PUR4_PHOPR,31.868,91,57,2,13,98,971,1061,2.400000e-02,40.0
4,MA_19953_MA_19953.g6.t1,gi|81960279|sp|Q914H8.1|Y054_SIFVH,19.880,166,122,4,17,175,176,337,9.800000e-02,37.7


### Test with big file

In [4]:
bb_file = '/Users/hcross/analysis/camel2016/its_refs/its_decomtamination/blout_itsUNK_to_nt.txt'

In [20]:
bb_paras = ['qseqid','sseqid','stitle','evalue','bitscore','qcovs','length','pident','nident','qstart','qend','sstart','send','qseq','sseq']

In [21]:
bb = blast_parser(bb_file, bb_paras)

In [32]:
bb_para1 = {'MIN_PCT_ID': 90, 'IN_TITLE': 'Uncultured', 'FORMAT': 'custom'}

In [23]:
bb1 = filter_blast(bb, bb_para1)

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [11]:
bb.stitle = bb.stitle.astype(str)

In [11]:
bb_para2 = {'IN_SUBJECT_ID': 'JX', 'FORMAT':'default'}

In [12]:
bb2 = filter_blast(bb, bb_para2)

In [13]:
bb2.head()

,qseqid,sseqid,stitle,evalue,bitscore,qcovs,length,pident,nident,qstart,qend,sstart,send,qseq,sseq
5,cam03.cf15_322,gi|410026385|gb|JX546605.1|,Uncultured fungus clone S9 18S ribosomal RNA g...,3.360000e-27,130.0,100,80,96.250,77,1,78,497,576,GCGGTGCGGCCAAGCCGTAAAACACCCCACTTCTTC--AAGGTTGA...,GCGGTGCGGCCACGCCGTAAAACACCCCACTTCTTCCAAAGGTTGA...
10,cam03.cf15_322,gi|404247867|gb|JX042888.1|,Uncultured fungus clone G2_DB7 18S ribosomal R...,1.560000e-25,124.0,99,77,96.104,74,1,77,498,573,GCGGTGCGGCCAAGCCGTAAAACACCCCACTTCTTCAAGGTTGACC...,GCGGTGCGGCCACGCCGTAAAACACCCCACTTC-TCAGGGTTGACC...
12,cam03.cf15_322,gi|395335538|gb|JX076982.1|,Acremonium chrysogenum strain GHL0901-3 intern...,2.020000e-24,121.0,87,68,98.529,67,1,68,404,471,GCGGTGCGGCCAAGCCGTAAAACACCCCACTTCTTCAAGGTTGACC...,GCGGTGCGGCCACGCCGTAAAACACCCCACTTCTTCAAGGTTGACC...
25,cam03.cf15_481,gi|410026385|gb|JX546605.1|,Uncultured fungus clone S9 18S ribosomal RNA g...,3.360000e-27,130.0,100,80,96.250,77,1,78,497,576,GCGGTGCGGCCAAGCCGTAAAACACCCCACTTCTTC--AAGGTTGA...,GCGGTGCGGCCACGCCGTAAAACACCCCACTTCTTCCAAAGGTTGA...
30,cam03.cf15_481,gi|404247867|gb|JX042888.1|,Uncultured fungus clone G2_DB7 18S ribosomal R...,1.560000e-25,124.0,99,77,96.104,74,1,77,498,573,GCGGTGCGGCCAAGCCGTAAAACACCCCACTTCTTCAAGGTTGACC...,GCGGTGCGGCCACGCCGTAAAACACCCCACTTC-TCAGGGTTGACC...


In [27]:
bb_para3 = {'MIN_PCT_ID': 90, 'IN_TITLE': 'Acremonium', 'FORMAT': 'custom'}

In [28]:
bb3 = filter_blast(bb, bb_para3)

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [18]:
qend1 = bb.ix[1,'stitle']
print(qend1)
print(type(qend1))

Ascomycota sp. AR-2010 isolate ATT243 18S ribosomal RNA gene, partial sequence; internal transcribed spacer 1, 5.8S ribosomal RNA gene, and internal transcribed spacer 2, complete sequence; and 28S ribosomal RNA gene, partial sequence
<type 'str'>


In [20]:
sseqid1 = bb.ix[1,'sseqid']
print(sseqid1)
print(type(sseqid1))

gi|312434386|gb|HQ607908.1|
<type 'str'>


In [21]:
bb_titles = bb['stitle'].tolist()

In [22]:
bb_titles[0]

'Acremonium sp. 904C internal transcribed spacer 1, partial sequence; 5.8S ribosomal RNA gene and internal transcribed spacer 2, complete sequence; and 28S ribosomal RNA gene, partial sequence'

In [23]:
type(bb_titles[0])

str

In [26]:
for x in bb_titles:
    tipo = type(x)
    if tipo != str:
        print x

In [28]:
bb[]

AttributeError: 'str' object has no attribute 'dtype'

**checking the dtype of all pandas columns**

http://stackoverflow.com/questions/41262370/determining-pandas-column-datatype

In [15]:
dtypeCount =[bb.iloc[:,i].apply(type).value_counts() for i in range(bb.shape[1])]

In [16]:
dtypeCount

[<type 'str'>    77624
 Name: qseqid, dtype: int64, <type 'str'>    77624
 Name: sseqid, dtype: int64, <type 'str'>    77624
 Name: stitle, dtype: int64, <type 'float'>    77624
 Name: evalue, dtype: int64, <type 'float'>    77624
 Name: bitscore, dtype: int64, <type 'int'>    77624
 Name: qcovs, dtype: int64, <type 'int'>    77624
 Name: length, dtype: int64, <type 'float'>    77624
 Name: pident, dtype: int64, <type 'int'>    77624
 Name: nident, dtype: int64, <type 'int'>    77624
 Name: qstart, dtype: int64, <type 'int'>    77624
 Name: qend, dtype: int64, <type 'int'>    77624
 Name: sstart, dtype: int64, <type 'int'>    77624
 Name: send, dtype: int64, <type 'str'>    77624
 Name: qseq, dtype: int64, <type 'str'>    77624
 Name: sseq, dtype: int64]

In [43]:
# so have to convert stitle (and others if necessary) to str
bb['stitle'] = bb['stitle'].astype(str)

In [47]:
# now run again
dtypeCount =[bb.iloc[:,i].apply(type).value_counts() for i in range(bb.shape[1])]

In [48]:
dtypeCount

[<type 'str'>    77624
 Name: qseqid, dtype: int64, <type 'str'>    77624
 Name: sseqid, dtype: int64, <type 'str'>    77624
 Name: stitle, dtype: int64, <type 'float'>    77624
 Name: evalue, dtype: int64, <type 'float'>    77624
 Name: bitscore, dtype: int64, <type 'int'>    77624
 Name: qcovs, dtype: int64, <type 'int'>    77624
 Name: length, dtype: int64, <type 'float'>    77624
 Name: pident, dtype: int64, <type 'int'>    77624
 Name: nident, dtype: int64, <type 'int'>    77624
 Name: qstart, dtype: int64, <type 'int'>    77624
 Name: qend, dtype: int64, <type 'int'>    77624
 Name: sstart, dtype: int64, <type 'int'>    77624
 Name: send, dtype: int64, <type 'str'>    77624
 Name: qseq, dtype: int64, <type 'str'>    77624
 Name: sseq, dtype: int64]

In [47]:
# try to reload file and convert in import
BB = pd.read_csv('/Users/hcross/analysis/camel2016/its_refs/its_decomtamination/blout_itsUNK_to_nt.txt', sep='\t', names=bb_paras, dtype={'stitle' : np.str })

In [48]:
bb_para1 = {'MIN_PCT_ID': 90, 'IN_TITLE': 'Uncultured', 'FORMAT': 'custom'}

In [49]:
BB1 = filter_blast(BB, bb_para1)

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [50]:
BB.head()

,qseqid,sseqid,stitle,evalue,bitscore,qcovs,length,pident,nident,qstart,qend,sstart,send,qseq,sseq
0,cam03.cf15_322,gi|564614030|gb|KF669512.1|,Acremonium sp. 904C internal transcribed space...,5.580000e-30,139.0,100,78,98.718,77,1,78,473,550,GCGGTGCGGCCAAGCCGTAAAACACCCCACTTCTTCAAGGTTGACC...,GCGGTGCGGCCACGCCGTAAAACACCCCACTTCTTCAAGGTTGACC...
1,cam03.cf15_322,gi|312434386|gb|HQ607908.1|,Ascomycota sp. AR-2010 isolate ATT243 18S ribo...,9.340000e-28,132.0,100,78,97.436,76,1,78,505,581,GCGGTGCGGCCAAGCCGTAAAACACCCCACTTCTTCAAGGTTGACC...,GCGGTGCGGCCACGCCGTAAAACACCCCACTTC-TCAAGGTTGACC...
2,cam03.cf15_322,gi|256855623|emb|FN397253.1|,Uncultured fungus genomic DNA sequence contain...,9.340000e-28,132.0,100,78,97.436,76,1,78,518,594,GCGGTGCGGCCAAGCCGTAAAACACCCCACTTCTTCAAGGTTGACC...,GCGGTGCGGCCACGCCGTAAAACACCCCACTTC-TCAAGGTTGACC...
3,cam03.cf15_322,gi|1030027947|gb|KU931422.1|,Uncultured fungus clone WJ139 18S ribosomal RN...,3.360000e-27,130.0,100,80,96.250,77,1,78,535,614,GCGGTGCGGCCAAGCCGTAAAACACCCCACTTCTTC--AAGGTTGA...,GCGGTGCGGCCACGCCGTAAAACACCCCACTTCTTCCAAAGGTTGA...
4,cam03.cf15_322,gi|1005260240|emb|HG937108.1|,Uncultured Acremonium genomic DNA containing 1...,3.360000e-27,130.0,94,73,98.630,72,6,78,520,592,GCGGCCAAGCCGTAAAACACCCCACTTCTTCAAGGTTGACCTCGGA...,GCGGCCACGCCGTAAAACACCCCACTTCTTCAAGGTTGACCTCGGA...


In [51]:
dtypeCount =[BB.iloc[:,i].apply(type).value_counts() for i in range(BB.shape[1])]

In [52]:
dtypeCount

[<type 'str'>    77624
 Name: qseqid, dtype: int64, <type 'str'>    77624
 Name: sseqid, dtype: int64, <type 'str'>    77624
 Name: stitle, dtype: int64, <type 'float'>    77624
 Name: evalue, dtype: int64, <type 'float'>    77624
 Name: bitscore, dtype: int64, <type 'int'>    77624
 Name: qcovs, dtype: int64, <type 'int'>    77624
 Name: length, dtype: int64, <type 'float'>    77624
 Name: pident, dtype: int64, <type 'int'>    77624
 Name: nident, dtype: int64, <type 'int'>    77624
 Name: qstart, dtype: int64, <type 'int'>    77624
 Name: qend, dtype: int64, <type 'int'>    77624
 Name: sstart, dtype: int64, <type 'int'>    77624
 Name: send, dtype: int64, <type 'str'>    77624
 Name: qseq, dtype: int64, <type 'str'>    77624
 Name: sseq, dtype: int64]

In [26]:
bb.stitle.apply(str)

0        Acremonium sp. 904C internal transcribed space...
1        Ascomycota sp. AR-2010 isolate ATT243 18S ribo...
2        Uncultured fungus genomic DNA sequence contain...
3        Uncultured fungus clone WJ139 18S ribosomal RN...
4        Uncultured Acremonium genomic DNA containing 1...
5        Uncultured fungus clone S9 18S ribosomal RNA g...
6        Uncultured Hapsidospora clone NG_M_G05 interna...
7        Uncultured Hapsidospora clone NG_M_A04 interna...
8        Acremonium chrysogenum rDNA internal transcrib...
9        Hypocreales sp. B1A0907P30CC473 internal trans...
10       Uncultured fungus clone G2_DB7 18S ribosomal R...
11       Uncultured fungus clone Puerto_Rico_OTULB_4308...
12       Acremonium chrysogenum strain GHL0901-3 intern...
13       Hypocreales sp. CY133 18S ribosomal RNA gene, ...
14       Uncultured fungus clone RFLP type 38 18S ribos...
15       Hypocreales sp. GMG_C1 18S ribosomal RNA gene,...
16       Sarcopodium circinosetiferum isolate PSN02 int.